In [ ]:
import os
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from config import password

In [ ]:
# Get api data
url = "https://api.covidtracking.com/v1/states/current.json"
response = requests.get(url).json()

In [ ]:
# Get data for  dataframe
# Determine the length of the response so we can use that for the number of states/territories
len(response)
num_state = range(0,56)
# Create empty lists
state = []
date = []
death = []
# Honestly, was super proud of this part bc i am BAD AT FOR LOOPS but Google/Zed Shaw is my religion now
# Create for loop to get the data that we need  for state, date, and death
for i in num_state:
    (print(response[i]["state"]))
    state.append(response[i]["state"])
    (print(response[i]["date"]))
    date.append(response[i]["date"])
    (print(response[i]["deathConfirmed"]))
    death.append(response[i]["death"])

In [ ]:
# Create covid dataframe ffrom api data
covid_deaths = pd.DataFrame({"state":state, "date": date, "death":death})
covid_deaths.head()

In [ ]:
# Import csv that includes data for Total Cases by state
# Filepath
csv_filepath = os.path.join("data","cdc_covid.csv")
# Read csv into DataFrame
covid_cases = pd.read_csv(csv_filepath)
# Review head and shape
covid_cases.head()
covid_cases.shape

In [ ]:
# Drop unnecessary columns from covid_cases
covid_cases = covid_cases.drop(columns = ["Confirmed Cases", "Total Deaths", "Probable Cases", "Cases in Last 7 Days", "Case Rate per 100000", "Confirmed Deaths", "Probable Deaths", "Deaths in Last 7 Days", "Death Rate per 100000", "Case Rate per 100000 in Last 7 Days", "Death Rate per 100K in Last 7 Days"])
# Review head
covid_cases.head()

In [ ]:
# Read in table to convert state name to state code
# Filepaths
state_csv = os.path.join("data","csvData.csv")
# Read csv into DataFrame
state_table = pd.read_csv(state_csv)

In [ ]:
state_table.head()

In [ ]:
covid_deaths_state = pd.merge(covid_deaths, state_table, left_on="state", right_on="Code", how="inner")

In [ ]:
covid_deaths_state.head()

In [ ]:
final_data = pd.merge(covid_deaths_state, covid_cases, left_on="State", right_on="State/Territory", how="inner")
final_data.head()

In [ ]:
# Limit the columns of the final data
final_data = final_data[["State/Territory", "date", "Code", "death", "Total Cases"]]
final_data.head()

In [ ]:
# Rename columns in preparation for Postgres connection
final_data = final_data.rename(columns = {"State/Territory": "state", "Total Cases": "total_cases", "Code": "code"})
final_data.head()

In [ ]:
# Convert date column to actual date
final_data['date'] = pd.to_datetime(final_data['date'].astype(str), format='%Y%m%d')
final_data.head()

In [ ]:
final_data.code.unique

In [ ]:
# Connect to database
pg_user = "postgres"
pg_password = password
db_name = "covid_ETL"
connection_string = f"{pg_user}:{password}@localhost:5432/{db_name}"
engine = create_engine(f"postgresql://{connection_string}")

In [ ]:
engine.table_names()

In [ ]:
final_data.to_sql(name="covid_table", con=engine, if_exists="append", index=False)